# Day 5: Supply Stacks

# Part 1

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked `crates`, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a `giant cargo crane` capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:

```
    [D]
[N] [C]
[Z] [M] [P]
 1   2   3

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```

In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

```
[D]
[N] [C]
[Z] [M] [P]
 1   2   3
```

In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:

```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
```

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
```

Finally, one crate is moved from stack 1 to stack 2:

```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
```

The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

**After the rearrangement procedure completes, what crate ends up on top of each stack?**


In [1]:
%load_ext nb_mypy

Version 1.0.4


In [ ]:
from __future__ import annotations

from collections import deque
from typing import Generator, Iterable, Generic, TypeVar

T = TypeVar("T")


class Stack(Generic[T]):
    _deque: deque
    black_list: list

    def __init__(
        self, items: Iterable[T] | None = None, black_list: list | None = None
    ) -> None:
        self._deque = deque(items) if items else deque()
        self.black_list = black_list or []

    def __len__(self) -> int:
        return len(self._deque)

    def __iadd__(self, other: Stack):
        self._deque += other._deque
        return self

    def __repr__(self) -> str:
        return f"Stack{repr(self._deque)[5:]}"

    def partition(self, quantity) -> Stack[T]:
        """Slices off and returns a new stack composed of the top `n` items."""
        partitioned_items = [self._deque.pop() for _ in range(quantity)]
        return Stack(reversed(partitioned_items))

    def append(self, item: T):
        self._deque.append(item)

    def add(self, item: T):
        if item in self.black_list:
            raise ValueError(f"{item} is on `black_list` - {self.black_list}")
        self.append(item)

    def pop(self) -> T:
        try:
            return self._deque.pop()
        except IndexError as empty_exc:
            raise IndexError("Stack is empty") from empty_exc

    @classmethod
    def _parse_line(cls, line: str) -> list[str]:
        return list(line[1::4])

    @classmethod
    def from_2d_array(
        cls, array: list[list[T]], black_list: list | None = None
    ) -> list[Stack[T]]:
        black_list = black_list or []
        # assume all rows are same length
        stacks = [cls(black_list=black_list) for _ in range(len(array[0]))]

        # starting from bottom row
        for row in array[::-1]:
            for idx, item in enumerate(row):
                if item not in black_list:
                    stacks[idx].add(item)
        return stacks

    @classmethod
    def from_diagram(cls, diagram_txt: str) -> list[Stack[str]]:
        _2d_array = [cls._parse_line(ln) for ln in diagram_txt.splitlines()]
        return cls.from_2d_array(_2d_array, black_list=[" "])  # type: ignore[arg-type,return-value]


diagram = """                [M]     [W] [M]    
            [L] [Q] [S] [C] [R]    
            [Q] [F] [F] [T] [N] [S]
    [N]     [V] [V] [H] [L] [J] [D]
    [D] [D] [W] [P] [G] [R] [D] [F]
[T] [T] [M] [G] [G] [Q] [N] [W] [L]
[Z] [H] [F] [J] [D] [Z] [S] [H] [Q]
[B] [V] [B] [T] [W] [V] [Z] [Z] [M]
"""

stacks = Stack.from_diagram(diagram)
display(stacks)  # type: ignore[name-defined]

print("Partion Stack\n-------------")
last_stack = stacks[-1]
print("  -Initial-", end="")
display(last_stack)

new_stack = last_stack.partition(2)
print("  -Final-")
display(last_stack, new_stack)


print("Combine Stacks\n---------------")
new_stack += last_stack
display(new_stack)

[Stack(['B', 'Z', 'T']),
 Stack(['V', 'H', 'T', 'D', 'N']),
 Stack(['B', 'F', 'M', 'D']),
 Stack(['T', 'J', 'G', 'W', 'V', 'Q', 'L']),
 Stack(['W', 'D', 'G', 'P', 'V', 'F', 'Q', 'M']),
 Stack(['V', 'Z', 'Q', 'G', 'H', 'F', 'S']),
 Stack(['Z', 'S', 'N', 'R', 'L', 'T', 'C', 'W']),
 Stack(['Z', 'H', 'W', 'D', 'J', 'N', 'R', 'M']),
 Stack(['M', 'Q', 'L', 'F', 'D', 'S'])]

Partion Stack
-------------
  -Initial-

Stack(['M', 'Q', 'L', 'F', 'D', 'S'])

  -Final-


Stack(['M', 'Q', 'L', 'F'])

Stack(['D', 'S'])

Combine Stacks
---------------


Stack(['D', 'S', 'M', 'Q', 'L', 'F'])

In [3]:
from typing import NamedTuple


class Instruction(NamedTuple):
    quantity: int
    source_idx: int
    target_idx: int

    @classmethod
    def parse_line(cls, line: str) -> Instruction:
        _, q, _, s, _, t = line.split(" ")
        return cls(int(q), int(s), int(t))


KT = TypeVar("KT", bound=int)
VT = TypeVar("VT", bound=Stack)


class CargoShip(Generic[KT, VT]):
    cargo: list[VT]

    @property
    def total_cargo(self):
        return sum(len(c) for c in self.cargo)

    def __init__(self, cargo: list[VT] | None = None):
        self.cargo = cargo or []

    def __len__(self) -> int:
        return len(self.cargo)

    def __repr__(self) -> str:
        return f"{type(self).__name__} |{'|'.join([str(len(c)) for c in self.cargo])}|"

    def __getitem__(self, idx: KT) -> VT:
        adjusted_idx = idx - 1
        num_cargo = len(self)
        if not idx or idx > num_cargo:
            raise IndexError(f"Incompatible `idx` {idx} expected 1 - {num_cargo}")
        return self.cargo[adjusted_idx]

    def move_cargo(self, source_idx: int, target_idx: int, quantity: int = 1):
        for _ in range(quantity):
            source_stack = self[source_idx]  # type: ignore[index]
            target_stack = self[target_idx]  # type: ignore[index]
            # print(f"{self}...\n{source_stack} -> {target_stack}")

            item = source_stack.pop()
            target_stack.add(item)
            # print(f"{source_stack} -> {target_stack}\n...{self}")

    def process_instruction(self, instruction: Instruction):
        initial_cargo = self.total_cargo
        self.move_cargo(
            instruction.source_idx, instruction.target_idx, instruction.quantity
        )
        assert self.total_cargo == initial_cargo


cs: CargoShip = CargoShip(stacks)
cs[1]
cs[9]

print(cs)
cs.process_instruction(Instruction(2, 1, 2))
print(cs)

CargoShip |3|5|4|7|8|7|8|8|4|
CargoShip |1|7|4|7|8|7|8|8|4|


In [4]:
from pathlib import Path
from typing import NamedTuple

INPUT_FILE = Path.cwd() / "inputs" / "day5.txt"
assert INPUT_FILE.exists(), INPUT_FILE


def extract_initial_cargo_stacks(input_file: Path) -> list[Stack]:
    starting_stacks: list[str] = []
    with open(input_file) as f_in:
        while True:
            line = next(f_in)
            if line == "\n":  # starting stacks section ends
                starting_stacks = starting_stacks[:-1]  # remove the unneeded stack labels
                return Stack.from_diagram("".join(starting_stacks))
            starting_stacks.append(line)


def extract_instructions(input_file: Path) -> Generator[Instruction, None, None]:
    with open(input_file) as f_in:
        line = next(f_in)
        while not line.startswith("\n"):
            line = next(f_in)
        while True:
            try:
                line = next(f_in)
                yield Instruction.parse_line(line)
            except StopIteration:
                break


stacks = extract_initial_cargo_stacks(INPUT_FILE)
ship: CargoShip = CargoShip(stacks)
print(f"Initial state\n{ship}")
ship.cargo

Initial state
CargoShip |3|5|4|7|8|7|8|8|6|


[Stack(['B', 'Z', 'T']),
 Stack(['V', 'H', 'T', 'D', 'N']),
 Stack(['B', 'F', 'M', 'D']),
 Stack(['T', 'J', 'G', 'W', 'V', 'Q', 'L']),
 Stack(['W', 'D', 'G', 'P', 'V', 'F', 'Q', 'M']),
 Stack(['V', 'Z', 'Q', 'G', 'H', 'F', 'S']),
 Stack(['Z', 'S', 'N', 'R', 'L', 'T', 'C', 'W']),
 Stack(['Z', 'H', 'W', 'D', 'J', 'N', 'R', 'M']),
 Stack(['M', 'Q', 'L', 'F', 'D', 'S'])]

In [5]:
for i, instr in enumerate(extract_instructions(INPUT_FILE), start=11):
    try:
        ship.process_instruction(instr)
    except IndexError as err:
        print(f"{instr} {i=} {err}")

print(f"\nFinal state\n{ship}")
ship.cargo


Final state
CargoShip |11|5|2|2|3|18|11|1|3|


[Stack(['J', 'C', 'L', 'D', 'R', 'G', 'T', 'L', 'M', 'H', 'N']),
 Stack(['Q', 'D', 'V', 'F', 'T']),
 Stack(['D', 'W']),
 Stack(['R', 'Z']),
 Stack(['J', 'M', 'Z']),
 Stack(['W', 'S', 'D', 'L', 'Q', 'F', 'Z', 'M', 'H', 'T', 'P', 'N', 'B', 'V', 'Z', 'B', 'V', 'W']),
 Stack(['D', 'G', 'T', 'G', 'N', 'Q', 'W', 'Q', 'S', 'F', 'H']),
 Stack(['F']),
 Stack(['S', 'M', 'V'])]

In [6]:
top_items: list[str] = []
for stack_number, stack in enumerate(ship.cargo, start=1):
    top = stack.pop()
    print(stack_number, top)
    top_items.append(top)
"".join(top_items)

1 N
2 T
3 W
4 Z
5 Z
6 W
7 H
8 F
9 V


'NTWZZWHFV'

## Part Two

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a CrateMover 9001.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and the ability to pick up and move multiple crates at once.

Again considering the example above, the crates begin in the same configuration:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
Moving a single crate from stack 2 to stack 1 behaves the same as before:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates stay in the same order, resulting in this new configuration:
```
        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3
```
Next, as both crates are moved from stack 2 to stack 1, they retain their order as well:

```
        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3
```
Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate C that gets moved:

```
        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3
```
In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. **After the rearrangement procedure completes, what crate ends up on top of each stack?**

In [7]:
class CrateMover9001(CargoShip):
    def move_cargo(self, source_idx: int, target_idx: int, quantity: int = 1):
        source_stack = self[source_idx]  # type: ignore[index]
        target_stack = self[target_idx]  # type: ignore[index]

        in_trasit_stack = source_stack.partition(quantity)
        target_stack += in_trasit_stack

In [8]:
# INPUT_FILE = INPUT_FILE.parent / "day5-p2.test"

stacks2 = extract_initial_cargo_stacks(INPUT_FILE)
ship2: CrateMover9001 = CrateMover9001(stacks2)
print(f"Initial state\n{ship2}")
ship2.cargo

Initial state
CrateMover9001 |3|5|4|7|8|7|8|8|6|


[Stack(['B', 'Z', 'T']),
 Stack(['V', 'H', 'T', 'D', 'N']),
 Stack(['B', 'F', 'M', 'D']),
 Stack(['T', 'J', 'G', 'W', 'V', 'Q', 'L']),
 Stack(['W', 'D', 'G', 'P', 'V', 'F', 'Q', 'M']),
 Stack(['V', 'Z', 'Q', 'G', 'H', 'F', 'S']),
 Stack(['Z', 'S', 'N', 'R', 'L', 'T', 'C', 'W']),
 Stack(['Z', 'H', 'W', 'D', 'J', 'N', 'R', 'M']),
 Stack(['M', 'Q', 'L', 'F', 'D', 'S'])]

In [9]:
for i, instr in enumerate(extract_instructions(INPUT_FILE), start=11):
    try:
        ship2.process_instruction(instr)
    except IndexError as err:
        print(f"{instr} {i=} {err}")

print(f"\nFinal state\n{ship2}")
ship2.cargo


Final state
CrateMover9001 |11|5|2|2|3|18|11|1|3|


[Stack(['D', 'Q', 'T', 'F', 'L', 'J', 'Q', 'V', 'M', 'M', 'B']),
 Stack(['Z', 'N', 'N', 'P', 'R']),
 Stack(['G', 'Z']),
 Stack(['L', 'G']),
 Stack(['V', 'H', 'F']),
 Stack(['D', 'W', 'M', 'F', 'H', 'S', 'L', 'Q', 'W', 'T', 'T', 'V', 'D', 'Z', 'D', 'W', 'H', 'V']),
 Stack(['D', 'F', 'W', 'G', 'S', 'R', 'M', 'Q', 'Z', 'S', 'B']),
 Stack(['T']),
 Stack(['N', 'C', 'J'])]

In [10]:
top_items: list[str] = []
for stack_number, stack in enumerate(ship2.cargo, start=1):
    top = stack.pop()
    top_items.append(top)
"".join(top_items)

'BRZGFVBTJ'